<a href="https://colab.research.google.com/github/MinorMole/RcloneLab/blob/master/RcloneLab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### 📚 For more information please visit our [GitHub](https://github.com/MinorMole/RcloneLab/).


# ![](https://minormole.github.io/RcloneLab/img/title_rclonelab.png =x45)

In [0]:
#@markdown <h3>📝 Note: Run this before use RcloneLab.</h3>
Setup_Time_Zone = True #@param {type:"boolean"}
Use_rclone_config_from_Google_Drive = False #@param {type:"boolean"}
rclone_Beta = False #@param {type:"boolean"}

import os
from IPython.display import clear_output
from google.colab import files
from datetime import datetime,timedelta

if Use_rclone_config_from_Google_Drive == True:
  try:
    print('Please follow the instructions below.\n')
    from google.colab import drive
    drive.mount('/GD')
    clear_output()
  except:
    clear_output()
    assert False, "Error occurred during mounting Google Drive."
else:
  try:
    !rm -rf /content/*
    print('Please upload the rclone.conf file from your computer.\n')
    uploaded = files.upload()
    for fn in uploaded.keys():
      print('User uploaded file "{name}" with length {length} bytes'.format(name=fn, length=len(uploaded[fn])))
    Config_File_Path = '/content/'+'{name}'.format(name=fn)
    clear_output()
  except:
    clear_output()
    assert False, "Error occurred during upload file."
if Setup_Time_Zone == True:
  !sudo dpkg-reconfigure tzdata
  clear_output()
if rclone_Beta == True:
  !curl https://rclone.org/install.sh | sudo bash -s beta
else:
  !curl https://rclone.org/install.sh | sudo bash
!mkdir /content/rclone_log/
clear_output()
print('Installation has been successfully completed.')

In [0]:
# ============================= FORM ============================= #
#@title <br><center>![](https://minormole.github.io/RcloneLab/img/title_rclonelab.png =x45)</center>
Mode = 'Config' #@param ["Config", "Copy", "Move", "Sync", "Checker", "Deduplicate", "Remove Empty Directories", "Empty Trash", "qBittorrent"]
Compare = 'Size & Mod-Time' #@param ["Size & Mod-Time", "Size & Checksum", "Only Mod-Time", "Only Size", "Only Checksum"]
Source = '' #@param {type:"string"}
Destination = '' #@param {type:"string"}
Transfers = 20 #@param {type:"slider", min:1, max:20, step:1}
Checkers = 40 #@param {type:"slider", min:1, max:40, step:1}
#@markdown ---
#@markdown <center><h3><font color="#3399ff"><b>⚙️ Global Configuration ⚙️</b></font></h3></center>
Simple_Ouput = True #@param {type:"boolean"}
Config = '' #@param {type:"string"}
#@markdown > <font color="red">* Required rclone.conf path. e.g. My Drive/rclone.conf (Only when using rclone.conf from Google Drive)</font>
Skip_files_that_are_newer_on_the_destination = False #@param {type:"boolean"}
Skip_all_files_that_exist = False #@param {type:"boolean"}
Do_not_cross_filesystem_boundaries = False #@param {type:"boolean"}
Do_not_update_modtime_if_files_are_identical = False #@param {type:"boolean"}
Large_amount_of_files_optimization = False #@param {type:"boolean"}
Google_Drive_optimization = False #@param {type:"boolean"}
Dry_Run = False #@param {type:"boolean"}
Output_Log = 'Disable' #@param ["Disable", "Download", "Google Drive (Use rclone.conf from Google Drive only)", "Local (/content/)"]
Output_Log_Level = 'NOTICE' #@param ["NOTICE", "INFO", "ERROR", "DEBUG"]
Extra_Arguments = '' #@param {type:"string"}
#@markdown - 📖 [Learn more...](https://rclone.org/commands/)
#@markdown ---
#@markdown <center><h3><font color="#3399ff"><b>↪️ Sync Configuration ↩️</b></font></h3></center>
Sync_Mode = 'Delete during transfer' #@param ["Delete during transfer", "Delete before transfering", "Delete after transfering"]
Track_Renames = False #@param {type:"boolean"}
#@markdown ---
#@markdown <center><h3><font color="#3399ff"><b>🔥 Deduplicate Configuration 🔥</b></font></h3></center>
Deduplicate_Mode = 'Interactive' #@param ["Interactive", "Skip", "First", "Newest", "Oldest", "Largest", "Rename"]
Deduplicate_Use_Trash = True #@param {type:"boolean"}
#@markdown - 📖 [Learn more...](https://rclone.org/commands/rclone_dedupe/)
# ================================================================ #

### Variable Declaration

# Optimized for Google Colaboratory
os.environ['bufferC'] = "--buffer-size 96M"

if Compare == 'Size & Checksum':
  os.environ['compareC'] = "-c"
elif Compare == 'Only Mod-Time':
  os.environ['compareC'] = "--ignore-size"
elif Compare == 'Only Size':
  os.environ['compareC'] = "--size-only"
elif Compare == 'Only Checksum':
  os.environ['compareC'] = "-c --ignore-size"
else:
  os.environ['compareC'] = ""

os.environ['sourceC'] = Source
os.environ['destinationC'] = Destination
os.environ['transfersC'] = '--transfers '+str(Transfers)
os.environ['checkersC'] = '--checkers '+str(Checkers)

if Use_rclone_config_from_Google_Drive == True:
  os.environ['configC'] = '/GD/'+Config
else:
  os.environ['configC'] = Config_File_Path

if Skip_files_that_are_newer_on_the_destination == True:
  os.environ['skipnewC'] = "-u"
else:
  os.environ['skipnewC'] = ""
  
if Skip_all_files_that_exist == True:
  os.environ['skipexistC'] = "--ignore-existing"
else:
  os.environ['skipexistC'] = ""
  
if Do_not_cross_filesystem_boundaries == True:
  os.environ['nocrossfilesystemC'] = "--one-file-system"
else:
  os.environ['nocrossfilesystemC'] = ""
  
if Do_not_update_modtime_if_files_are_identical == True:
  os.environ['noupdatemodtimeC'] = "--no-update-modtime"
else:
  os.environ['noupdatemodtimeC'] = ""

if Large_amount_of_files_optimization == True:
  os.environ['filesoptimizeC'] = "--fast-list"
else:
  os.environ['filesoptimizeC'] = ""
  
if Google_Drive_optimization == True:
  os.environ['driveoptimizeC'] = "--drive-chunk-size 32M --drive-acknowledge-abuse --drive-keep-revision-forever"
else:
  os.environ['driveoptimizeC'] = ""
  
if Dry_Run == True:
  os.environ['dryrunC'] = "-n"
else:
  os.environ['dryrunC'] = ""
  
if Output_Log != "Disable":
  os.environ['statsC'] = "--log-file=rclone_log.txt"
else:
  if Simple_Ouput == True:
    os.environ['statsC'] = "-v --stats-one-line --stats=5s"
  else:
    os.environ['statsC'] = "-v --stats=5s"
  
if Output_Log_Level == "INFO" and Output_Log != "Disable":
  os.environ['loglevelC'] = "--log-level INFO"
elif Output_Log_Level == "ERROR" and Output_Log != "Disable":
  os.environ['loglevelC'] = "--log-level ERROR"
elif Output_Log_Level == "DEBUG" and Output_Log != "Disable":
  os.environ['loglevelC'] = "--log-level DEBUG"
else:
  os.environ['loglevelC'] = ""

os.environ['extraC'] = Extra_Arguments

if Sync_Mode == 'Delete during transfer':
  os.environ['syncmodeC'] = "--delete-during"
elif Sync_Mode == 'Delete before transfering':
  os.environ['syncmodeC'] = "--delete-before"
elif Sync_Mode == 'Delete after transfering':
  os.environ['syncmodeC'] = "--delete-after"
  
if Track_Renames == True:
  os.environ['trackrenamesC'] = "--track-renames"
else:
  os.environ['trackrenamesC'] = ""
  
if Deduplicate_Mode == 'Interactive':
  os.environ['deduplicateC'] = "interactive"
elif Deduplicate_Mode == 'Skip':
  os.environ['deduplicateC'] = "skip"
elif Deduplicate_Mode == 'First':
  os.environ['deduplicateC'] = "first"
elif Deduplicate_Mode == 'Newest':
  os.environ['deduplicateC'] = "newest"
elif Deduplicate_Mode == 'Oldest':
  os.environ['deduplicateC'] = "oldest"
elif Deduplicate_Mode == 'Largest':
  os.environ['deduplicateC'] = "largest"
elif Deduplicate_Mode == 'Rename':
  os.environ['deduplicateC'] = "rename"
  
if Deduplicate_Use_Trash == True:
  os.environ['deduplicatetrashC'] = ""
else:
  os.environ['deduplicatetrashC'] = "--drive-use-trash=false"
  
### rclone Execution

if Output_Log != "Disable" and Mode != "Config":
  print("Logging enables, rclone will not output log through the terminal, please wait until finished.")

if Config == '' and Use_rclone_config_from_Google_Drive == True:
  print("Please config the rclone.conf in Setup section first.")
elif Mode == 'Config':
  !rclone --config="$configC" config
elif Mode == 'Copy':
  !rclone --config="$configC" copy "$sourceC" "$destinationC" $transfersC $checkersC $statsC $loglevelC $compareC $skipnewC $skipexistC $nocrossfilesystemC $noupdatemodtimeC $bufferC $filesoptimizeC $driveoptimizeC $dryrunC $extraC
elif Mode == 'Move':
  !rclone --config="$configC" move "$sourceC" "$destinationC" $transfersC $checkersC $statsC $loglevelC --delete-empty-src-dirs $compareC $skipnewC $skipexistC $nocrossfilesystemC $noupdatemodtimeC $bufferC $filesoptimizeC $driveoptimizeC $dryrunC $extraC
elif Mode == 'Sync':
  !rclone --config="$configC" sync "$sourceC" "$destinationC" $transfersC $checkersC $statsC $loglevelC $syncmodeC $trackrenamesC $compareC $skipnewC $skipexistC $nocrossfilesystemC $noupdatemodtimeC $bufferC $filesoptimizeC $driveoptimizeC $dryrunC $extraC
elif Mode == 'Checker':
  !rclone --config="$configC" check "$sourceC" "$destinationC" $checkersC $statsC $loglevelC $compareC $skipnewC $skipexistC $nocrossfilesystemC $noupdatemodtimeC $bufferC $filesoptimizeC $driveoptimizeC $dryrunC $extraC
elif Mode == 'Deduplicate':
  !rclone --config="$configC" dedupe "$sourceC" $checkersC $statsC $loglevelC --dedupe-mode $deduplicateC $deduplicatetrashC $compareC $skipnewC $skipexistC $nocrossfilesystemC $noupdatemodtimeC $bufferC $filesoptimizeC $driveoptimizeC $dryrunC $extraC
elif Mode == 'Remove Empty Directories':
  !rclone --config="$configC" rmdirs "$sourceC" $statsC $loglevelC $dryrunC $extraC
elif Mode == 'Empty Trash':
  !rclone --config="$configC" cleanup "$sourceC" $statsC $loglevelC $dryrunC $extraC
elif Mode == 'qBittorrent':
  !rclone --config="$configC" move "/root/qBittorrent/" "$destinationC" $transfersC $checkersC $statsC $loglevelC --delete-empty-src-dirs $compareC $skipnewC $skipexistC $nocrossfilesystemC $noupdatemodtimeC $bufferC $filesoptimizeC $driveoptimizeC $dryrunC $extraC

### Log Output
  
if Output_Log != "Disable" and Mode != "Config":
  ### Rename log file and output settings.
  print('\nOutputting log file...')
  !mv '/content/rclone_log.txt' /content/rclone_log/rclone_log_$(date +%Y-%m-%d_%H.%M.%S).txt
  with open('/content/rclone_log/'+Mode+'_settings.txt', 'w') as f:
    f.write('Mode: ' + Mode + '\n' \
            'Compare: ' + Compare + '\n' \
            'Source: \"' + Source + '\"\n' \
            'Destination: \"' + Destination + '\"\n' \
            'Transfers: ' + str(Transfers) + '\n' \
            'Checkers: ' + str(Checkers) + '\n' \
            'Skip files that are newer on the destination: ' + str(Skip_files_that_are_newer_on_the_destination) + '\n' \
            'Skip all files that exist: ' + str(Skip_all_files_that_exist) + '\n' \
            'Do not cross filesystem boundaries: ' + str(Do_not_cross_filesystem_boundaries) + '\n' \
            'Do not update modtime if files are identical: ' + str(Do_not_update_modtime_if_files_are_identical) + '\n' \
            'Dry-Run: ' + str(Dry_Run) + '\n' \
            'Output Log Level: ' + Output_Log_Level + '\n' \
            'Extra Arguments: \"' + Extra_Arguments + '\"\n' \
            'Sync Moden: ' + Sync_Mode + '\n' \
            'Track Renames: ' + str(Track_Renames) + '\n' \
            'Deduplicate Mode: ' + Deduplicate_Mode + '\n' \
            'Deduplicate Use Trash: ' + str(Deduplicate_Use_Trash))
  ### Compressing log file.
  print('Compressing log file...')
  !zip -r -q -j -9 '/content/rclone_log.zip' '/content/rclone_log/'
  !rm -rf '/content/rclone_log/'
  !mkdir '/content/rclone_log/'
  ### Send log to the user desire destination.
  if Output_Log == "Local (/content/)":
    !mv '/content/rclone_log.zip' /content/rclone_log_$(date +%Y-%m-%d_%H.%M.%S).zip
    print('Your log file locate at \"/content/\", you can use file explorer of Google Colaboratory to find and download it.')
  elif Output_Log == "Google Drive (Use rclone.conf from Google Drive only)" and Use_rclone_config_from_Google_Drive == True:
    os.environ['logpathC'] = "/".join(Config.split("/")[:-1])
    !mv '/content/rclone_log.zip' /GD/"$logpathC"/rclone_log_$(date +%Y-%m-%d_%H.%M.%S).zip
    !echo Your log will be ready at \"$logpathC\"
  else:
    print('Your log file will be download soon...')
    files.download('/content/rclone_log.zip')
    !rm '/content/rclone_log.zip'

### Operation has been successfully completed.
if Mode != "Config":
  print('\nOperation has been successfully completed.')

# ![](https://minormole.github.io/RcloneLab/img/title_qbittorrent.png =x45)

In [0]:
# ============================= FORM ============================= #
#@markdown <h3>📝 Note: Run this cell and relax.</h3>
Install_File_Server = False #@param {type:"boolean"}
# ================================================================ #

import time
import string, random
from IPython.display import clear_output

def random_generator(size=8, chars=string.ascii_lowercase + string.digits):
  return ''.join(random.choice(chars) for x in range(size))
GenLink = random_generator()
!pkill -9 -e -f qbittorrent-nox
!pkill -9 -e -f ssh
clear_output()
!yes '' | add-apt-repository ppa:qbittorrent-team/qbittorrent-stable
clear_output()
!apt-get update
clear_output()
!apt-get install qbittorrent-nox
clear_output()
!mkdir -p /root/{qBittorrent,qBittorrent_Temp}
!rm /root/.config/qBittorrent/qBittorrent.conf
!wget "https://minormole.github.io/RcloneLab/conf/qBittorrent.conf" -P "/root/.config/qBittorrent/"
clear_output()
get_ipython().system_raw("printf 'y' | qbittorrent-nox -d --webui-port=6006 &")
get_ipython().system_raw("ssh -fNT -o StrictHostKeyChecking=no -R " + GenLink + ":80:localhost:6006 serveo.net &")
clear_output()
if Install_File_Server == True:
  GenHTTPLink = random_generator()
  !rm /root/MultithreadedSimpleHTTPServer.py
  !rm /root/qBittorrent/favicon.ico
  !wget "https://minormole.github.io/RcloneLab/conf/MultithreadedSimpleHTTPServer.py" -P "/root/"
  !wget "https://minormole.github.io/RcloneLab/conf/favicon.ico" -P "/root/qBittorrent/"
  !chmod a+x /root/MultithreadedSimpleHTTPServer.py
  get_ipython().system_raw("ssh -fNT -o StrictHostKeyChecking=no -R " + GenHTTPLink + ":80:localhost:7007 serveo.net &")
  get_ipython().system_raw("python /root/MultithreadedSimpleHTTPServer.py 0.0.0.0:7007 /root/qBittorrent/ &")
  HTTPMsg = "Website: https://" + GenHTTPLink + ".serveo.net\n"
else:
  HTTPMsg = ""
clear_output()

for x in range(30, 0, -1):
    print("\r Startup in progress... %d sec" % x, end="")
    time.sleep(1)
clear_output()
print("If WebUI return blank, please press enter at the address bar of the WebUI tab (Don't use F5 key).\n" \
      "If it still not responding, please rerun this cell.\n\n" + HTTPMsg + "WebUI: https://" + GenLink + \
      ".serveo.net\nUsername: rclonelab\nPassword: rclonelab")

# ![](https://minormole.github.io/RcloneLab/img/title_utility.png =x45)

In [0]:
# ============================= FORM ============================= #
#@markdown <h3>Check VM's Status</h3>
Loop_Check = False #@param {type:"boolean"}
Loop_Interval = 5 #@param {type:"slider", min:1, max:30, step:1}
# ================================================================ #

import time
from IPython.display import clear_output
Loop = True

while Loop == True:
  clear_output()
  !top -n 1
  if Loop_Check == False:
    Loop = False
  else:
    time.sleep(Loop_Interval)

In [0]:
# ============================= FORM ============================= #
#@markdown <h3>Get VM's Specification</h3>
Output_Format = 'TEXT' #@param ["TEXT", "HTML", "XML", "JSON"]
Short_Output = False #@param {type:"boolean"}
# ================================================================ #

import os
from google.colab import files
from IPython.display import clear_output

if Short_Output == True:
  os.environ['outputformatC'] = "txt"
  os.environ['outputformat2C'] = "-short"
  Output_Format_Ext = 'txt'
elif Output_Format == "TEXT":
  os.environ['outputformatC'] = "txt"
  os.environ['outputformat2C'] = ""
  Output_Format_Ext = 'txt'
else:
  os.environ['outputformatC'] = Output_Format.lower()
  os.environ['outputformat2C'] = "-"+Output_Format.lower()
  Output_Format_Ext = Output_Format.lower()
  
!apt-get install lshw
clear_output()
!lshw $outputformat2C > Specification.$outputformatC
try:
  files.download('/content/Specification.'+Output_Format_Ext)
  print('Your specification report file will be download soon...')
except:
  clear_output()
  assert False, "Error occurred during download file."
!rm /content/Specification.$outputformatC